In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pandas as pd
import html5lib
from bs4 import BeautifulSoup
import re
import math
from random import randint
import os
from time import sleep

# CHANGE TITLE HERE

In [2]:
title = 'teacher'

# Run Behind Proxies

In [3]:
#driver.close()

In [4]:
phantom_path = '/usr/local/bin/phantomjs'

service_args = ['--proxy=proxy-nl.privateinternetaccess.com:1080', 
                '--proxy-type=socks5', 
                '--proxy-auth=x6122622:rtQokhs2Dd',
                '--web-security=false',
                '--ignore-ssl-errors=true']

driver = webdriver.PhantomJS(phantom_path, service_args=service_args)

In [5]:
driver.get('https://secure.indeed.com/account/login?service=rex&hl=en_US&co=US&cfb=2&continue=http%3A%2F%2Fwww.indeed.com%2Fresumes')

In [ ]:
driver.set_window_size(1080,720)
try:
    username_form = driver.find_element_by_id('signin_email')
    username_form.send_keys("bryantbiggs@gmail.com")
    username_form.send_keys(Keys.TAB)
except Exception:
    driver.save_screenshot('email.png')
    driver.close()

In [ ]:
driver.set_window_size(1920,1080)
try:
    password_form = driver.find_element_by_id('signin_password')
    password_form.send_keys("Yamaharx1!")
    password_form.send_keys(Keys.ENTER)
except Exception:
    driver.save_screenshot('password.png')
    driver.close()

In [ ]:
driver.save_screenshot('logged.png')

In [ ]:
_ = '''
import requests
import socket
import socks

PORT = 1080
# no authentication
# socks.set_default_proxy(socks.SOCKS5, "localhost")
# with Authentication
socks.set_default_proxy(socks.SOCKS5, "proxy-nl.privateinternetaccess.com",PORT,True,"x6122622","ZzQCgNVhum")
socket.socket = socks.socksocket
'''

# Show Current IP

In [ ]:
def print_ip():
    try:
        url = 'http://www.showmyip.gr/'
        driver.get(url)

        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        # get tag with ip address
        ip_tag = soup.find(attrs={'class': 'ip_address'})
        ip = (re.findall(r'''([0-9.]+)''', str(ip_tag)))[0]
        return ip
    except:
        return '#'*30

In [ ]:
print_ip()

# Get Number of Pages for Title

In [ ]:
def num_results(title):
    url = 'http://www.indeed.com/resumes?q={0}&co=US'.format(title)
    driver.get(url)
    
    # get page with selenium and pass to beautiful soup
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # get number of results for query
    result_count = soup.find(attrs={'id': 'result_count'})

    try:
        # grab number only
        num = (re.findall(r'''([0-9,]+)''', str(result_count)))[0]
        num = int(num.replace(',', ''))
        return num
    except:
        return 0

In [ ]:
num_results(title)

# Navigate to Subsequent Pages

In [ ]:
_ = '''
def navigate(start_page):
# return web page source for scraping names
    try:
        next_button = driver.find_element_by_xpath('//a[@href="{0}"]'.format('?q=c&co=US&start={0}'.format(str(start_page))))
        next_button.click()
    except:
        pass
    
    #sleep(5)
    html = driver.page_source
    
    return BeautifulSoup(html, 'lxml')
'''

# Make List of Links

In [ ]:
def make_links(title):
    list_links = []
    list_links.append('http://www.indeed.com/resumes?q={0}&co=US'.format(title))

    for start_page in range(50, num_results(title), 50):
        list_links.append('http://www.indeed.com/resumes?q={0}&co=US&start={1}'.format(title, str(start_page)))

    return list_links

In [ ]:
list_names = make_links(title)
#list_names = list_names[20:]

In [ ]:
list_names[0]

In [ ]:
def get_users(url):
    temp = []
    ct = 0
    while len(temp) == 0:
        if ct >= 5: break
        try:
            driver.get(url)
            driver.save_screenshot('pics/{0}.png'.format(url[-15:]))
            ct += 1
            ### SLEEP
            sleep(randint(2,8))
            ### SLEEP
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
            
            # find users
            list_of_people = soup.find_all(attrs={'class':'clickable_resume_card'})
            # make list of users found
            temp = [re.search(r'''(\/.+)\?''', person.get('onclick')).group() for person in list_of_people]
        except:
            continue

    return temp

In [ ]:
def scrape_users(title):
    # cleanup previous file versions before starting due to append
    f = '{0}.csv'.format(title)
    if os.path.exists(f): os.remove(f)
    if os.path.exists(f): os.remove('failed.csv')
    
    ttl = 0
    ct = 0
    # loop through pages
    for name in list_names:
            # make list of users found
            temp = get_users(name)

            # show what ip is being used
            ttl += len(temp)
            print('List: {0} --- {1}'.format(ttl, print_ip()))
            
            
            # write list of names found to file
            title = title.replace(' ', '_')
            with open('{0}.csv'.format(title), 'a') as outfile:
                for name in temp:
                    outfile.write('{0}, \n'.format(name))
            ct += 1
            if ct % 3 == 0:
                driver.delete_all_cookies()

In [ ]:
def failed_page(title, start_page):
    with open('failed.csv', 'a') as outfile:
        outfile.write(('title:{0}, page_start:{1}, \n').format(title, start_page))

In [ ]:
scrape_users(title)